# End-to-end Multi-Class Dog Breed Classification

An image classifier using TensorFlow and TensorFlow Hub.

## 1. Problem

Pretty straigt forward. If I have a picture of a dog, I want to know what breed of dog it is.

## 2. Data

The data is from Kaggle's dog breed identification [competition](https://www.kaggle.com/competitions/dog-breed-identification/data)

## 3. Evaluation

[Evaluation metrics](https://www.kaggle.com/competitions/dog-breed-identification/overview/evaluation): evaluated on Multi-class Log Loss

## 4. Features

* I'm dealing with images (unstructured data)
  - good candidate for deep learning/transfer learning
* 10,000+ images in training set
  - labels
* 10,000+ images in test set
  - no labels

## Prepare Workspace

- import tf and tf hub
- set up GPU

In [1]:
import pandas as pd
import numpy as np
# tensorflow import
import tensorflow as tf
import tensorflow_hub as hub
print("tf version: ", tf.__version__)
print("tf hub version: ", hub.__version__)

# check for gpu availability
print("GPU", "Available :)" if tf.config.list_physical_devices("GPU") else "Not Available :(")

tf version:  2.14.0
tf hub version:  0.15.0
GPU Available :)


In [1]:
# unzip data into googleDrive
# %cd /content/drive/MyDrive/tf_breed_classification/data/dog-vision-real

# !unzip "/content/drive/MyDrive/tf_breed_classification/data/dog-breed-identification.zip" -d "/content/drive/MyDrive/tf_breed_classification/data/dog-vision-real"


In [3]:
!pwd

/content


In [ ]:
%cd test/

/content/drive/MyDrive/tf_breed_classification/data/dog-vision-real/test


In [ ]:
!ls